In [62]:
import json
import pandas as pd
import numpy as np
import random
def dump_json(data, file_name):
    with open(f'./{file_name}.json', 'w') as fout:
        json.dump(data, fout, cls=DateTimeEncoder)
        
def read_json(file_name):
    with open(f'./{file_name}.json', 'r') as fout:
        return json.load(fout)

In [63]:
round_one = read_json('claims_first_round.csv')
round_two = read_json('claims_second_round')
round_three = read_json('claims_third_round')

In [64]:
tweet_dict = {}

def fill_dict(round_):
    misses = 0
    hits = 0
    max_answer_length_found = 0
    for i, hit in enumerate(round_):
        key = hit['tweet']
        
        max_answer_length_found = max(len(hit['answers']), max_answer_length_found)
        if key in tweet_dict:
            hits = hits + 1
            workers = {}
            for answ in [*tweet_dict[key]['answers'], *hit['answers']]:
                worker = answ['worker']
                if worker not in workers:
                    workers[worker] = answ
            max_answer_length_found = max(len(workers.values()), max_answer_length_found)
            tweet_dict[key]['answers'] = list(workers.values())
            
        else:
            tweet_dict[key] = hit.copy()
            misses = misses + 1
            
    print(misses, hits, max_answer_length_found)
            
fill_dict(round_one)
fill_dict(round_two)
fill_dict(round_three)
results = list(tweet_dict.values())

387 3 3
0 390 7
0 304 7


In [65]:
len(results)

387

In [66]:
dist = {}
for item in results:
    key = len(item['answers'])
    if key in dist:
        dist[key] = dist[key] + 1
    else:
        dist[key] = 1
        
dist

{5: 186, 6: 38, 4: 99, 3: 60, 7: 4}

In [67]:
results[0]['answers']

[{'QuestionFormAnswers': {'@xmlns': 'http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd',
   'Answer': {'QuestionIdentifier': 'claim',
    'FreeText': 'Plant based food should be encouraged|Vegan and vegetarian diets should be encouraged'}},
  'worker': 'A3CFNUD7VR2E1E',
  'answer': 'Plant based food should be encouraged|Vegan and vegetarian diets should be encouraged'},
 {'QuestionFormAnswers': {'@xmlns': 'http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd',
   'Answer': {'QuestionIdentifier': 'claim',
    'FreeText': 'Plant based food should be encouraged'}},
  'worker': 'ABY1RK8H6HC8C',
  'answer': 'Plant based food should be encouraged'},
 {'QuestionFormAnswers': {'@xmlns': 'http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd',
   'Answer': {'QuestionIdentifier': 'claim',
    'FreeText': 'Plant based food should be encouraged'}},
  'worke

In [68]:
claims = []
for item in results:
    answers = item['answers']
    tweet = item['tweet']
    topic_dist = {}
    for answer in answers:
        keys = answer['answer'].split('|')
        for key in keys:
            if key in topic_dist:
                topic_dist[key] = topic_dist[key] + 1
            else:
                topic_dist[key] = 1

    claims.append((tweet, topic_dist, len(answers)))
        

In [69]:
claims[155]

('Vegetarian maybe but not vegan',
 {'Vegan and vegetarian diets should be encouraged': 4,
  'Plant based food should be encouraged': 2,
  'no claim': 1},
 7)

In [70]:
#for (tweet, topic_dist, sup) in arguments:
#    row = df[df.tweet == tweet]
#    df.loc[df.tweet == tweet, 'topic'] = (arg_dist['argumentative'] if 'argumentative' in arg_dist else 0)/sum(arg_dist.values())
df = pd.read_csv('argumentative.csv').drop(columns=['Unnamed: 0', 'index'])
df = pd.DataFrame(df_rec).sample(frac=1).drop_duplicates(subset=['id'])
topics_en_v2 = [
    "We should reduce the consumption of meat",
    "Plant based food should be encouraged",
    "Meat alternatives should be encouraged",
    "Vegan and vegetarian diets should be encouraged",
    "We should pursue policies that promote sustainable foods"
]

df_rec = []
for r in df.to_dict('records'):
    for topic in topics_en_v2:
        rec = r.copy()
        rec['topic'] = {}
        rec['topic'] = topic
        rec['arg_type'] = ''
        rec['unrelated_prob'] = 0
        df_rec.append(rec)
df = pd.DataFrame(df_rec)

In [74]:
for i, (tweet, d, num_answers) in enumerate(claims):        
    nc = 0

    if 'no claim' in d:
        nc = d['no claim']/num_answers
    df.loc[df.tweet == tweet, 'arg_type'] = 'unrelated'
    df.loc[df.tweet == tweet, 'unrelated_prob'] = 1
    
    for topic in d:
        if topic != 'no claim':
            select = (df.tweet == tweet) & (df.topic == topic)
            df.loc[select,'claim'] = d[topic]/num_answers
            df.loc[select, 'arg_type'] = ''
            df.loc[select, 'unrelated_prob'] = nc
      

In [333]:
df_rec = []
for r in df.to_dict('records'):
    topics = r['topic'].copy()
    if len(topics) > 0:
        for topic in topics:
            rec = r.copy()
            nc = 0
            
            if 'no claim' in topics:
                nc = topics['no claim']
            
            if topic != 'no claim':
                rec['claim'] = topics[topic]
                rec['topic'] = topic
                rec['unrelated_prob'] = nc
                df_rec.append(rec)
            
        left_over_topics = np.intersect1d(list(r['topic'].keys()), topics_en_v2)    
        if len(left_over_topics) > 0:
            r['topic'] = random.choice(left_over_topics)
            r['arg_type'] = 'unrelated'
            df_rec.append(r)
    else: 
        r['topic'] = random.choice(topics_en_v2)
        df_rec.append(r)
        

In [334]:
df = pd.DataFrame(df_rec)

In [78]:
df.to_csv('claims-large.csv', index = False)

In [341]:
sum(df.claim >= 0.5)

289